In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 파일이 있는 폴더로 이동

In [43]:
cd /content/drive/MyDrive/vgg

/content/drive/MyDrive/vgg


# 데이터셋 압축 해제

In [ ]:
!unzip -qq datasets.zip

Archive:  datasets.zip
   creating: datasets/
  inflating: datasets/.DS_Store      
  inflating: __MACOSX/datasets/._.DS_Store  
   creating: datasets/test/
   creating: datasets/train/
   creating: datasets/val/
   creating: datasets/test/dogs/
   creating: datasets/test/cats/
  inflating: datasets/test/.DS_Store  
  inflating: __MACOSX/datasets/test/._.DS_Store  
   creating: datasets/train/dogs/
   creating: datasets/train/cats/
  inflating: datasets/train/.DS_Store  
  inflating: __MACOSX/datasets/train/._.DS_Store  
   creating: datasets/val/dogs/
   creating: datasets/val/cats/
  inflating: datasets/val/.DS_Store  
  inflating: __MACOSX/datasets/val/._.DS_Store  
  inflating: datasets/test/dogs/dog.4039.jpg  
  inflating: datasets/test/dogs/dog.4005.jpg  
  inflating: datasets/test/dogs/dog.4011.jpg  
  inflating: datasets/test/dogs/dog.4010.jpg  
  inflating: datasets/test/dogs/dog.4004.jpg  
  inflating: datasets/test/dogs/dog.4038.jpg  
  inflating: datasets/test/dogs/dog.4012

# 라이브러리 불러오기

In [44]:
from tqdm import tqdm

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

# device 정의

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# 데이터 전처리

In [46]:
# ImageFolder를 통해 데이터셋 생성
def Dataset(root):
    transform = transforms.Compose( [
                        transforms.Resize(250),
                        transforms.CenterCrop(224),

                        transforms.ToTensor(),
                        transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])])

    return torchvision.datasets.ImageFolder(root=root, transform=transform)

In [47]:
data_dir = "./datasets"
batch_size = 16

train_dataset = Dataset(root=data_dir+"/train")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = Dataset(root=data_dir+"/val")
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

test_dataset = Dataset(root=data_dir+"/test")
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)

# Model 작성

In [48]:
VGG_types = {
    'VGG11' : [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13' : [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

In [49]:
class Conv(nn.Module):
    def __init__(self, in_channels, architecture):
        super(Conv, self).__init__()

        layers = []

        for x in VGG_types[architecture]:
            if type(x) == int:
                out_channels = x

                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                     kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]

        self.conv_layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.conv_layers(x)

In [50]:
class FC(nn.Module):
    def __init__(self, num_classes):
        super(FC, self).__init__()

        if num_classes == 1:
            self.fc_layers = nn.Sequential(
                nn.Linear(512*7*7, 4096),
                nn.ReLU(),
                nn.Dropout(p=0.5),
                nn.Linear(4096, 4096),
                nn.ReLU(),
                nn.Dropout(p=0.5),
                nn.Linear(4096, num_classes),
                nn.Sigmoid()
            )
        else:
            self.fc_layers = nn.Sequential(
                nn.Linear(512*7*7, 4096),
                nn.ReLU(),
                nn.Dropout(p=0.5),
                nn.Linear(4096, 4096),
                nn.ReLU(),
                nn.Dropout(p=0.5),
                nn.Linear(4096, num_classes)
            )

    def forward(self, x):
        return self.fc_layers(x)

In [51]:
class VGG(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000, architecture="VGG16"):
        super(VGG, self).__init__()
        self.num_classes = num_classes

        self.conv_layers = Conv(in_channels=in_channels, architecture=architecture)
        if num_classes == 2:
            self.fc_layers = FC(num_classes=1)
        else:
            self.fc_layers = FC(num_classes=num_classes)
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc_layers(x)
        return x

# 모델 선언

In [52]:
num_epochs = 10
lr = 1e-4

model = VGG(in_channels=3, num_classes=len(train_dataset.classes), architecture="VGG16").to(device)

# 손실함수와 옵티마이저 정의

In [53]:
if len(train_dataset.classes) == 2:
    fn_loss = nn.BCELoss()
else:
    fn_loss = nn.CrossEntropyLoss()

optim = torch.optim.Adam(model.parameters(), lr=lr)

# 모델 학습

In [55]:
start = 0
for epoch in range(start+1, num_epochs+1):
    print("Epoch {}/{}".format(epoch, num_epochs))
    print("-"*10)
    model.train()

    running_loss = 0.0
    running_correct = 0

    for inputs, outputs in tqdm(train_loader):

        inputs = inputs.to(device)
        labels = outputs.to(device)

        if len(train_dataset.classes) == 2:
            outputs = model(inputs).squeeze()

            optim.zero_grad()

            loss = fn_loss(outputs, labels.to(torch.float32))
            loss.backward()

            optim.step()

            outputs[outputs>=0.5] = 1
            outputs[outputs<0.5] = 0
        else:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            optim.zero_grad()

            loss = fn_loss(outputs, labels)
            loss.backward()

            outputs = preds

            optim.step()


        running_loss += loss.item() * batch_size
        running_correct += torch.sum(outputs == labels.data).to(int)


    epoch_loss = running_loss / train_size
    epoch_acc = running_correct.double() / train_size

    print("Train Loss: {:.4f}\tAccuracy: {:.4f}".format(epoch_loss, epoch_acc))

    running_loss = 0.0
    running_correct = 0

    with torch.no_grad():
        model.eval()

        for inputs, outputs in tqdm(val_loader):
            inputs = inputs.to(device)
            labels = outputs.to(device)

            if len(train_dataset.classes) == 2:
                outputs = model(inputs).squeeze()
                loss = fn_loss(outputs, labels.to(torch.float32))

                outputs[outputs>=0.5] = 1
                outputs[outputs<0.5] = 0
            else:
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = fn_loss(outputs, labels)

                outputs = preds

            running_loss += loss.item() * batch_size
            running_correct += torch.sum(outputs == labels.data)

        epoch_loss = running_loss / val_size
        epoch_acc = running_correct.double() / val_size

        print("Valid Loss: {:.4f}\tAccuracy: {:.4f}".format(epoch_loss, epoch_acc))

        torch.save(model, "./model.pth")

Epoch 1/10
----------


100%|██████████| 75/75 [00:29<00:00,  2.50it/s]


Train Loss: 1.0484	Accuracy: 0.5325


100%|██████████| 10/10 [00:01<00:00,  5.79it/s]


Valid Loss: 0.7823	Accuracy: 0.5467
Epoch 2/10
----------


...

Valid Loss: 0.6562	Accuracy: 0.6400
Epoch 10/10
----------


100%|██████████| 75/75 [00:29<00:00,  2.54it/s]


Train Loss: 0.5718	Accuracy: 0.7125


100%|██████████| 10/10 [00:01<00:00,  6.52it/s]


Valid Loss: 0.6030	Accuracy: 0.6867


# 모델 Test

In [56]:
model = torch.load("./model.pth").to(device)

running_loss = 0.0
running_correct = 0

with torch.no_grad():
    model.eval()

    for inputs, outputs in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = outputs.to(device)

        if len(test_dataset.classes) == 2:
            outputs = model(inputs).squeeze()
            loss = fn_loss(outputs, labels.to(torch.float32))
            outputs[outputs>=0.5] = 1
            outputs[outputs<0.5] = 0
        else:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = fn_loss(outputs, labels)
            outputs = preds

        running_loss += loss.item() * batch_size
        running_correct += torch.sum(outputs == labels.data)

    epoch_loss = running_loss / test_size
    epoch_acc = running_correct.double() / test_size
    print("Test Loss: {:.4f}\tAccuracy: {:.4f}".format(epoch_loss, epoch_acc))

100%|██████████| 10/10 [00:01<00:00,  5.82it/s]

Test Loss: 0.6142	Accuracy: 0.6933
